In [12]:
#Imports

import csv
import re

Functions:

In [13]:
#Functions
def extract_sections(input_string):
    
    subjective_reg = r'SUBJECTIVE.*?(?=OBJECTIVE|ASSESSMENT|PLAN|$)'
    objective_reg = r'OBJECTIVE.*?(?=SUBJECTIVE|ASSESSMENT|PLAN|$)'
    assessment_reg = r'ASSESSMENT.*?(?=OBJECTIVE|SUBJECTIVE|PLAN|$)'
    plan_reg = r'PLAN.*?(?=SUBJECTIVE|ASSESSMENT|OBJECTIVE|$)'

    subjective_matches = re.findall(subjective_reg, input_string)
    objective_matches = re.findall(objective_reg, input_string)
    assessment_matches = re.findall(assessment_reg, input_string)
    plan_matches = re.findall(plan_reg, input_string)

    return subjective_matches, objective_matches, assessment_matches, plan_matches

def remove_brackets(strings_list):
    pattern = r'\[|\]'  # Regular expression pattern matching '[' or ']'
    cleaned_list = [re.sub(pattern, '', string) for string in strings_list]
    return cleaned_list

def convert_nested_list_to_strings(lst):
    strings_list = [str(sublist) if sublist else '' for sublist in lst]
    return remove_brackets(strings_list)

def add_lists(list1, list2):
    list1.extend(list2)
    return list1

def process_transcription_semantics(transcription):
    transcription = transcription.replace(",", "")
    transcription = transcription.replace('"', "")
    transcription = transcription.replace("'", "")
    return transcription

    

Main Part

In [14]:

filename = "temporary_datasets/final_dataset_raw.csv"
file = open(filename,"r", encoding='utf-8')
data = list(csv.reader(file,delimiter=";"))
file.close()


#add SOAP headers
header = data[0]
header.extend(['Subjective', 'Objective', 'Assessment', 'Plan'])

content = []
output=[]
full_row = []
patient_id = []


for sublist in data[1:]:

    second_value = sublist[0:]
    transcription = second_value[3]

    patient_id = [second_value[0]]
    description = [second_value[1]]
    medical_speciality = [second_value[2]]

    transcription = process_transcription_semantics(transcription)

    sbj, obj, ass, plan = extract_sections(transcription)


    if len(sbj) == 0:
        sbj.append("NONE")

    if len(obj) == 0:
        obj.append("NONE")
    
    if len(ass) == 0:
        ass.append("NONE")

    if len(plan) == 0:
        plan.append("NONE")



    soap_notes = []
    soap_notes.extend(sbj)
    soap_notes.extend(obj)
    soap_notes.extend(ass)
    soap_notes.extend(plan)

    
    single_row = patient_id + description + medical_speciality + [second_value[3]] + soap_notes

    full_row.append(single_row)














Write CSV file

In [15]:
with open('temporary_datasets/final_dataset.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(full_row)